# SPOC Code Generation (Colab GPU)

Generate code on GPU, then download results for local evaluation.

## Setup

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# Clone repo (first time only)
!rm -rf spoc
!git clone https://github.com/tlebryk/spoc.git
# %cd YOUR_REPO/spoc

Cloning into 'spoc'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 3), reused 28 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 24.15 KiB | 4.02 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
# Install dependencies
!pip install -q transformers torch accelerate pandas tqdm sacrebleu azure-storage-blob

In [ ]:
# Download SPOC data (if not in repo)
!wget https://sumith1896.github.io/spoc/data/spoc.zip
!unzip -q spoc.zip && mv spoc/* . && rm -rf spoc spoc.zip

--2025-11-25 04:47:48--  https://sumith1896.github.io/spoc/data/spoc.zip
Resolving sumith1896.github.io (sumith1896.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to sumith1896.github.io (sumith1896.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9960355 (9.5M) [application/x-zip-compressed]
Saving to: ‘spoc.zip.1’

spoc.zip.1          100%[===================>]   9.50M  --.-KB/s    in 0.07s   

2025-11-25 04:47:48 (138 MB/s) - ‘spoc.zip.1’ saved [9960355/9960355]

replace test/spoc-testw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Load Modules

In [ ]:
import os
from google.colab import userdata

os.environ["AZURE_STORAGE_CONNECTION_STRING"] = userdata.get("AZURE_STORAGE_CONNECTION_STRING")

In [ ]:
import sys
sys.path.insert(0, './src')

from data_loader import SPOCDataLoader
from inference import CodeGenerator

## Generate Code (GPU)

In [ ]:
!mv ../../test/ ../

In [ ]:
%cd /content/spoc/scripts

/content/spoc/scripts


In [ ]:
!  python run_eval.py \
    --run-name exp1 \
    --model Qwen/Qwen2.5-Coder-0.5B \
    --split testp \
    --n-samples 2


SPOC Code Generation Evaluation

Model: Qwen/Qwen2.5-Coder-0.5B
Split: testp
N samples: 2
Temperature: 0.2
Using hidden test cases

✓ Connected to Azure storage
Run name: exp1

--------------------------------------------------------------------------------
Loading data...
--------------------------------------------------------------------------------
✓ Loaded 1815 programs from testp
✓ Using 2 programs for evaluation

--------------------------------------------------------------------------------
Generating code...
--------------------------------------------------------------------------------
Loading model: Qwen/Qwen2.5-Coder-0.5B
Device: cuda
tokenizer_config.json: 7.23kB [00:00, 31.1MB/s]
vocab.json: 2.78MB [00:00, 72.5MB/s]
merges.txt: 1.67MB [00:00, 129MB/s]
tokenizer.json: 7.03MB [00:00, 174MB/s]
config.json: 100% 659/659 [00:00<00:00, 6.91MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
2025-11-25 04:50:39.765369: E external/local_xla/xla/stream_executor/cuda/cuda_fft

In [ ]:
!ls outputs

ls: cannot access 'outputs': No such file or directory


In [ ]:
"""Example usage of Azure Blob Storage."""

import sys
from pathlib import Path
# sys.path.insert(0, str(Path(__file__).parent.parent / "src"))

from azure_storage import AzureStorage

# Initialize (reads AZURE_STORAGE_CONNECTION_STRING from environment)
storage = AzureStorage(container="spoc")

# Save evaluation results
results = {
    "timestamp": "20250124_120000",
    "metrics": {"CompAcc": 0.85, "FCorrAcc": 0.72},
    "programs": ["program1", "program2"]
}
# storage.save_json(results, "outputs/eval_20250124.json")

# Load results

loaded = storage.load_json("outputs/eval_20250124.json")

# Save a file
storage.save_file("outputs/metrics.txt", "outputs/metrics_20250124.txt")

# Download a file
storage.load_file("outputs/metrics_20250124.txt", "local_metrics.txt")

# List all evaluation results
eval_files = storage.list(prefix="outputs/eval_")
print(f"Found {len(eval_files)} evaluation results")


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/metrics.txt'

In [ ]:
!pip install azure-storage-blob

In [ ]:
from azure.storage.blob import BlobServiceClient



In [ ]:
loaded